In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy import optimize

# coin toss example (complete information)

In [2]:
m = 10 # number of flips in experiment
n = 5 # number of experiments

p_1 = 0.8
p_2 = 0.3

xs = [] # number of heads in each experiment
zs = [0,0,1,0,1] # which coin to flip

np.random.seed(5)
for i in zs:
    if i==0:
        xs.append(stats.binom(n=m, p=p_1).rvs()) # flip coin 1
    elif i==1:
        xs.append(stats.binom(n=m, p=p_2).rvs()) # flip coin 2

In [3]:
xs = np.array(xs)
print(xs)

[9 7 2 6 3]


### analytical solution

In [4]:
exp1 = [0,1,3] # indexes of experiments with coin 1
exp2 = [2,4] # indexes of experiments with coin 2

print('Analytical solutions:')
print('p1: ', xs[exp1].sum() / (m * len(exp1)))
print('p2: ', xs[exp2].sum() / (m * len(exp2)))

Analytical solutions:
p1:  0.7333333333333333
p2:  0.25


### numerical solution

In [5]:
def neg_log_likelihood(probs, m, xs, zs):
    '''
    compute negative log-likelihood
    '''
    ll = 0
    for x,z in zip(xs,zs):
        ll += stats.binom(p=probs[z], n=m).logpmf(x)
        
    return -ll

In [6]:
res = optimize.minimize(neg_log_likelihood, [0.5,0.5], bounds=[(0,1),(0,1)], args=(m,xs,zs), method='tnc')

In [7]:
print('Numerical solution:')
print('p1: ', res.x[0])
print('p2: ', res.x[1])

Numerical solution:
p1:  0.7333327749188117
p2:  0.24999991839920377


# EM algo (example from the paper)

In [8]:
m = 10 # number of flips in each sample
n = 5 # number of samples
xs = np.array([5,9,8,4,7])

theta = [0.6, 0.5] # initial guess p_1, p_2

for i in range(10):
    p_1,p_2 = theta
    T_1s = []
    T_2s = []
    
    # E-step
    for x in xs:
        T_1 = stats.binom(n=m,p=theta[0]).pmf(x) / (stats.binom(n=m,p=theta[0]).pmf(x) + 
                                                    stats.binom(n=m,p=theta[1]).pmf(x))
        T_2 = stats.binom(n=m,p=theta[1]).pmf(x) / (stats.binom(n=m,p=theta[0]).pmf(x) + 
                                                    stats.binom(n=m,p=theta[1]).pmf(x))
        
        T_1s.append(T_1)
        T_2s.append(T_2)
        
    
    # M-step
    T_1s = np.array(T_1s)
    T_2s = np.array(T_2s)
    p_1 = np.dot(T_1s, xs) / (m * np.sum(T_1s))
    p_2 = np.dot(T_2s, xs) / (m * np.sum(T_2s))
    print(f'step:{i}, p1={p_1:.2f}, p2={p_2:.2f}')
    theta = [p_1, p_2]

step:0, p1=0.71, p2=0.58
step:1, p1=0.75, p2=0.57
step:2, p1=0.77, p2=0.55
step:3, p1=0.78, p2=0.53
step:4, p1=0.79, p2=0.53
step:5, p1=0.79, p2=0.52
step:6, p1=0.80, p2=0.52
step:7, p1=0.80, p2=0.52
step:8, p1=0.80, p2=0.52
step:9, p1=0.80, p2=0.52


# EM algo (Binomial mixture)

In [23]:
# model parameters
p_1 = 0.1
p_2 = 0.8
tau_1 = 0.3
tau_2 = 1-tau_1

m = 10 # number of flips in each sample
n = 10 # number of samples

# generate data
np.random.seed(123)
dists = [stats.binom(n=m, p=p_1), stats.binom(n=m, p=p_2)]
xs = [dists[x].rvs() for x in np.random.choice([0,1], size=n, p=[tau_1,tau_2])]

In [24]:
print(xs)

[9, 1, 1, 10, 8, 7, 9, 9, 8, 8]


In [26]:
# random initial guess
np.random.seed(123)
theta = [np.random.rand() for _ in range(3)]

last_ll = 0
max_iter = 100

for i in range(max_iter):
    tau,p_1,p_2 = theta
    T_1s = []
    T_2s = []
    
    # E-step
    lls = []
    for x in xs:
        denom = (tau * stats.binom(n=m,p=p_1).pmf(x) + (1-tau) * stats.binom(n=m,p=p_2).pmf(x))
        T_1 = tau * stats.binom(n=m,p=p_1).pmf(x) / denom
        T_2 = (1-tau) * stats.binom(n=m,p=p_2).pmf(x) / denom
        T_1s.append(T_1)
        T_2s.append(T_2)
        lls.append(T_1 * np.log(tau * stats.binom(n=m,p=p_1).pmf(x)) + 
                   T_2 * np.log(tau * stats.binom(n=m,p=p_2).pmf(x)))
    
    # M-step
    T_1s = np.array(T_1s)
    T_2s = np.array(T_2s)
    tau = np.sum(T_1s) / n
    p_1 = np.dot(T_1s, xs) / (m * np.sum(T_1s))
    p_2 = np.dot(T_2s, xs) / (m * np.sum(T_2s))
    print(f'step:{i}, tau={tau:.2f}, p1={p_1:.2f}, p2={p_2:.2f}, log_likelihood={sum(lls):.2f}')
    theta = [tau, p_1, p_2]
    
    # stop when likelihood doesn't improve anymore
    if abs(sum(lls) - last_ll) < 0.001:
        break
    else:
        last_ll=sum(lls)

step:0, tau=0.86, p1=0.75, p2=0.38, log_likelihood=-74.14
step:1, tau=0.80, p1=0.85, p2=0.11, log_likelihood=-20.71
step:2, tau=0.80, p1=0.85, p2=0.10, log_likelihood=-14.88
step:3, tau=0.80, p1=0.85, p2=0.10, log_likelihood=-14.83
step:4, tau=0.80, p1=0.85, p2=0.10, log_likelihood=-14.83


### bigger number of samples

In [20]:
p_1 = 0.1
p_2 = 0.8
tau_1 = 0.3
tau_2 = 1-tau_1

m = 10 # number of flips in each sample
n = 200 # number of samples

# generate data
np.random.seed(123)
dists = [stats.binom(n=m, p=p_1), stats.binom(n=m, p=p_2)]
xs = [dists[x].rvs() for x in np.random.choice([0,1], size=n, p=[tau_1,tau_2])]

In [21]:
print(xs)

[8, 0, 1, 4, 7, 8, 10, 7, 8, 10, 7, 8, 8, 2, 8, 8, 0, 0, 7, 9, 8, 5, 9, 6, 6, 10, 8, 1, 0, 10, 0, 10, 9, 7, 6, 10, 9, 5, 8, 8, 10, 1, 5, 9, 10, 1, 6, 9, 7, 7, 0, 8, 7, 8, 9, 5, 8, 7, 10, 9, 8, 10, 5, 9, 9, 0, 10, 0, 0, 8, 0, 9, 10, 9, 1, 9, 8, 0, 1, 10, 6, 10, 9, 7, 10, 7, 5, 2, 7, 6, 8, 7, 9, 6, 9, 6, 2, 7, 0, 8, 10, 7, 0, 0, 9, 7, 9, 8, 8, 9, 8, 8, 0, 8, 8, 6, 8, 6, 1, 8, 10, 1, 2, 9, 10, 8, 1, 7, 10, 6, 7, 2, 8, 7, 9, 1, 8, 2, 9, 9, 3, 2, 9, 8, 9, 9, 9, 0, 6, 7, 1, 6, 1, 9, 9, 0, 9, 10, 8, 9, 8, 9, 8, 0, 9, 7, 8, 9, 8, 7, 6, 1, 9, 7, 7, 0, 2, 1, 7, 9, 9, 3, 1, 7, 7, 9, 1, 7, 8, 10, 0, 9, 6, 8, 0, 9, 0, 8, 9, 2]


In [22]:
# random initial guess
np.random.seed(123)
theta = [np.random.rand() for _ in range(3)]

last_ll = 0
max_iter = 100

for i in range(max_iter):
    tau,p_1,p_2 = theta
    T_1s = []
    T_2s = []
    
    # E-step
    lls = []
    for x in xs:
        denom = (tau * stats.binom(n=m,p=p_1).pmf(x) + (1-tau) * stats.binom(n=m,p=p_2).pmf(x))
        T_1 = tau * stats.binom(n=m,p=p_1).pmf(x) / denom
        T_2 = (1-tau) * stats.binom(n=m,p=p_2).pmf(x) / denom
        T_1s.append(T_1)
        T_2s.append(T_2)
        lls.append(T_1 * np.log(tau * stats.binom(n=m,p=p_1).pmf(x)) + 
                   T_2 * np.log(tau * stats.binom(n=m,p=p_2).pmf(x)))
    
    # M-step
    T_1s = np.array(T_1s)
    T_2s = np.array(T_2s)
    tau = np.sum(T_1s) / n
    p_1 = np.dot(T_1s, xs) / (m * np.sum(T_1s))
    p_2 = np.dot(T_2s, xs) / (m * np.sum(T_2s))
    print(f'step:{i}, tau={tau}, p1={p_1:.2f}, p2={p_2:.2f}, likelihood={sum(lls):.2f}')
    theta = [tau, p_1, p_2]
    
    # stop when likelihood doesn't improve anymore
    if abs(sum(lls) - last_ll) < 0.001:
        break
    else:
        last_ll=sum(lls)

step:0, tau=0.8349444847603371, p1=0.69, p2=0.31, likelihood=-1319.73
step:1, tau=0.7476959535947724, p1=0.80, p2=0.10, likelihood=-482.42
step:2, tau=0.7526128303772595, p1=0.80, p2=0.09, likelihood=-373.30
step:3, tau=0.7538000667944199, p1=0.80, p2=0.08, likelihood=-371.20
step:4, tau=0.7539575693865782, p1=0.80, p2=0.08, likelihood=-370.89
step:5, tau=0.7539776112051868, p1=0.80, p2=0.08, likelihood=-370.85
step:6, tau=0.7539801481913413, p1=0.80, p2=0.08, likelihood=-370.84
step:7, tau=0.7539804691184753, p1=0.80, p2=0.08, likelihood=-370.84
